# Setup 

In [ ]:
!python --version

In [ ]:
# !pip install openai

In [ ]:
from openai import AzureOpenAI

In [ ]:
url_base = 'https://ubc-cv-sherlock.s3.us-west-2.amazonaws.com/oops_val_v1_frames/'
subsets = ['preevent', 'postevent', 'event']

## OpenAI calls

In [ ]:
from keys import *
GPT4V_KEY=GPT4_KEY


client = AzureOpenAI(
      api_version="2024-02-15-preview",
      azure_endpoint="https://culturalsnapsweden.openai.azure.com/",
      api_key=GPT4V_KEY)


headers = {
    "Content-Type": "application/json",
    "api-key": GPT4V_KEY,
}


In [ ]:
def get_response(messages):
  response = client.chat.completions.create(
      # engine="gpt4v", # engine = "deployment_n
      model="gpt4",
      messages= messages,
    #   response_format="json",
      max_tokens=500,
  )
  return response

In [ ]:
def evaluate(ground_truth_texts, model_texts):
    ###  ground truth
    gold = "\n".join([f"{i+1}. {item['Explanation']}" for i, item in enumerate(ground_truth_texts)])
    #predicted = "\n".join([f"{i+1}. {item['Explanation']}" for i, item in enumerate(model_texts)])
    predicted = f"1. {model_texts[0]['Explanation']}"


    task1_prompt = f"""You will be given given 3 ground truth (gt) descriptions of a video (set1) and a single descriptions to evaluate (set2).

Your task is to compare set2 against set1 and rate it based on two metrics:

1. Specificity: Measures the extent to which the set2 description includes detailed and distinguishing features present in set1/ground truth description.

2. Similarity: Measures how closely the content of the set2 description matches the content of the set1/ground truth description. This metric evaluates the overlap in information, phrasing, and key elements between the two descriptions.

Both metrics need to be scored on a scale of 1-5. 

Return a python dictionary, containing two keys, Specificity and Similarity. The value of each key is a python tuple containing the score out of 5 and the reason for the score. 

Here are the ground truth (set1) descriptions: 
{gold}
Here is the evaluation (set2) description:
{predicted}
"""
    system_prompt = "You are an expert in evaluating text generated from AI models."
    messages = []
    messages.append({"role": "system", "content": [
            {
                "type": "text",
                "text": system_prompt
            },
            ],})

    messages.append({"role": "user", "content": [

            {
                "type": "text",
                "text":  task1_prompt
            },
            ],})

    res1 = get_response(messages)
    messages.append({'role': res1.choices[0].message.role, 'content': res1.choices[0].message.content})

    return messages, res1.choices[0].message.content


In [ ]:
main_prompt = """You are a masterful English language grader. Your expertise is used to evaluate video descriptions against a set of reference descriptions. Your goal is to evaluate the video description based on the following:

1. Semantic similarity: measures how closely the meaning of the video description aligns with the reference descriptions. It focuses on the overall context and key actions/events described.
To evaluate this, you will rate each of the following from 1-5, 1 being low and 5 being high:
(a) Action/Event Matching: Identify and compare the main actions or events described.
(b) Context and Outcome: Assess if the context and outcomes described in the video description are similar to those in the reference descriptions.
(c) Use of Similar Words/Phrases: Note the use of similar or synonymous words and phrases.

2. Level of detail: measures how detailed and specific the video description is compared to the reference descriptions. It focuses on the richness of the description and the number of distinct details provided, and not the meaning of the content itself.
To evaluate this, you will rate each of the following from 1-5:
(a) Detail Level: Count the number of specific details and descriptive elements. If the number of details in the video description match the number of details in the reference prompts, then give it a three. If more, give it a 4 or 5, and if lower, give it a 1 or 2 rating.
(b) Clarity of Actions/Events: Evaluate how clearly and specifically actions/events are described. If the clarity matches that of the reference descriptions, then rate it a 3. If clarity is higher, you can rate it 4 or 5, and if lower, rate it 1 or 2.

Grade the video description, and explain your evaluation. Your output should be in JSON, following this template:
{"Action/Event Matching" : {"Rating": X, "Explanation": "..."}...
"""

def evaluate(ground_truth_texts, model_texts):
    ###  ground truth
    gold = "\n".join([f"{i+1}. {item['Explanation']}" for i, item in enumerate(ground_truth_texts)])
    #predicted = "\n".join([f"{i+1}. {item['Explanation']}" for i, item in enumerate(model_texts)])
    results = []
    for i in range(0, 3):
        predicted = f"{model_texts[i]['Explanation']}"

        task1_prompt = f"""Here's a video description:
        {predicted}

        Here's a reference set of descriptions:
        {gold}"""

        system_prompt = main_prompt
        messages = []
        messages.append({"role": "system", "content": [
                {
                    "type": "text",
                    "text": system_prompt
                },
                ],})

        messages.append({"role": "user", "content": [

                {
                    "type": "text",
                    "text":  task1_prompt
                },
                ],})

        res1 = get_response(messages)
        results.append(res1.choices[0].message.content)
        messages.append({'role': res1.choices[0].message.role, 'content': res1.choices[0].message.content})
    
    results = '['+','.join(results)+']'

    return messages, results


In [ ]:
main_prompt = """You are a masterful English language grader. Your expertise is used to evaluate video descriptions against a set of reference descriptions. Your goal is to evaluate the video description based on the following:

2. Level of detail: measures how detailed and specific the video description is compared to the reference descriptions. It focuses on the richness of the description and the number of distinct details provided, and not the meaning of the content itself.
To evaluate this, you will rate each of the following from 1-5:
(a) Detail Level: Count the number of specific details and descriptive elements. If the number of details in the video description match the number of details in the reference prompts, then give it a three. If more, give it a 4 or 5, and if lower, give it a 1 or 2 rating.
(b) Clarity of Actions/Events: Evaluate how clearly and specifically actions/events are described. If the clarity matches that of the reference descriptions, then rate it a 3. If clarity is higher, you can rate it 4 or 5, and if lower, rate it 1 or 2.

Grade the video description, and explain your evaluation. Your output should be in JSON, following this template:
{"Action/Event Matching" : {"Rating": X, "Explanation": "..."}...
"""

def evaluate(ground_truth_texts, model_texts):
    ###  ground truth
    gold = "\n".join([f"{i+1}. {item['Explanation']}" for i, item in enumerate(ground_truth_texts)])
    #predicted = "\n".join([f"{i+1}. {item['Explanation']}" for i, item in enumerate(model_texts)])
    results = []
    for i in range(0, 3):
        predicted = f"{model_texts[i]['Explanation']}"

        task1_prompt = f"""Here's a video description:
        {predicted}

        Here's a reference set of descriptions:
        {gold}"""

        system_prompt = main_prompt
        messages = []
        messages.append({"role": "system", "content": [
                {
                    "type": "text",
                    "text": system_prompt
                },
                ],})

        messages.append({"role": "user", "content": [

                {
                    "type": "text",
                    "text":  task1_prompt
                },
                ],})

        res1 = get_response(messages)
        results.append(res1.choices[0].message.content)
        messages.append({'role': res1.choices[0].message.role, 'content': res1.choices[0].message.content})
    
    results = '['+','.join(results)+']'

    return messages, results


In [ ]:
from openai import AzureOpenAI
#original data
import json
import pandas as pd

## Inference on gpt4 - text only


In [ ]:

def extract_overall_json(json_string):
    """
    Extracts JSON from the given string and processes it.
    """
    try:
        if pd.isna(json_string):
            return None
        json_string = json_string.replace("json", "").replace("```", "").strip()
        json_string = json.loads(json_string)
        return json_string
    except (json.JSONDecodeError, TypeError, KeyError, IndexError):
        return json_string

In [ ]:

json_file = "oops_val_v3_mf_2.json"
with open(json_file, 'r') as f:
    data = json.load(f)
    


In [ ]:

from tqdm import tqdm
def process_json_file(data):

    final_responses = []

    # Open the JSON file in append mode
    for item in tqdm(data[:5]):
        task1_responses = item['task1_responses']
        # print(task1_responses)
        if task1_responses and len(task1_responses) > 1:
            # image_urls = [url_base + f'{item[sub][0:-4]}/frame_{i}.jpg' for sub in subsets for i in range(1, 5)]
        
            all_messages, task1_responses = evaluate(item['task1_gt'], item['task1_responses'])
            # print(task1_responses)
            fr = item
            # Add responses to the new item
            fr['metric'] = task1_responses

            final_responses.append(fr)

    return final_responses

In [ ]:
new_data = process_json_file(data)

In [ ]:
for item in new_data:
    item['metric'] = extract_overall_json(item['metric'])

In [ ]:
#Write the new data to a new JSON file
with open(json_file.replace(".json", "_metric.json"), 'w') as f:
    json.dump(new_data, f, indent=4,  separators=(',', ':'))

In [ ]:
from IPython.display import Video, HTML
def video_display(idx):
    video_link = f"https://ubc-cv-sherlock.s3.us-west-2.amazonaws.com/oops/oops_val_v3/{idx}"

    # Embed the video using HTML
    video_html = f"<video controls src='{video_link}' width='500' height='400'>"
    display(HTML(video_html))

In [ ]:
import pprint
for item in new_data:
    video_display(item['preevent'])
    print("HUMAN:")
    pprint.pp(item['task1_gt'])
    print("AI:")
    pprint.pp(item['task1_responses'][0]['Explanation'])
    pprint.pp(item['metric'])
    print("#########################")

In [ ]:
import pprint
for item in new_data:
    video_display(item['preevent'])
    print("HUMAN:")
    pprint.pp(item['task1_gt'])
    print("AI:")
    pprint.pp(item['task1_responses'])
    pprint.pp(item['metric'])
    print("#########################")

# OpenAI Assistant API

# Doesn't work well...

In [1]:
from openai import OpenAI
client = OpenAI()
from keys import *
GPT4V_KEY=GPT4_KEY


client = AzureOpenAI(
      api_version="2024-05-01-preview",
      azure_endpoint="https://culturalsnapsweden.openai.azure.com/",
      api_key=GPT4V_KEY)


headers = {
    "Content-Type": "application/json",
    "api-key": GPT4V_KEY,
}

assistant = client.beta.assistants.create(
    instructions="""You are a masterful English language grader. Your expertise is used to evaluate video descriptions against a set of reference descriptions. Your goal is to evaluate the video description based on the following:
            1. Semantic similarity: measures how closely the meaning of the video description aligns with the reference descriptions. It focuses on the overall context and key actions/events described.
            To evaluate this, you will rate each of the following from 1-5:
            (a) Action/Event Matching: Identify and compare the main actions or events described.
            (b) Context and Outcome: Assess if the context and outcomes described in the video description are similar to those in the reference descriptions.
            (c) Use of Similar Words/Phrases: Note the use of similar or synonymous words and phrases.

            2. Level of detail: measures how detailed and specific the video description is compared to the reference descriptions. It focuses on the richness of the description and the number of distinct details provided, not the content itself..
            To evaluate this, you will rate each of the following from 1-5:
            (a) Detail Level: Count the number of specific details and descriptive elements.
            (b) Clarity of Actions/Events: Evaluate how clearly and specifically actions/events are described.
            
            When asked to grade, your output should always be in JSON, following this template:
            {"Action/Event Matching" : {"Rating": X, "Explanation": "..."}...
            """,
    name="English Language Grader",
    model="gpt4",
    response_format={ "type": "json_object" }
)
print(assistant)

NameError: name 'AzureOpenAI' is not defined

In [ ]:
def evaluate_assistant(ground_truth_texts, model_text):
  reference = "\n".join([f"{i+1}. {item['Explanation']}" for i, item in enumerate(ground_truth_texts)])

  thread = client.beta.threads.create()
  message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=f"""Here's a video description:
    {model_text}

    Here's a reference set of descriptions:
    {reference}

    Grade it and respond in JSON only.""")

  run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=assistant.id
  )

  #print(client.beta.threads.messages.data)
  if run.status == 'completed': 
    messages = client.beta.threads.messages.list(
      thread_id=thread.id
    )
    #print(messages._get_page_items())
    return messages._get_page_items()[0].content
  else:
    print(run.status)

In [ ]:
from tqdm import tqdm
import json
def process_json_file(data):

    final_responses = []

    # Open the JSON file in append mode
    for item in tqdm(data[:5]):
        task1_responses = item['task1_responses']
        # print(task1_responses)
        if task1_responses and len(task1_responses) > 1:
            # image_urls = [url_base + f'{item[sub][0:-4]}/frame_{i}.jpg' for sub in subsets for i in range(1, 5)]
        
            task1_responses = evaluate_assistant(item['task1_gt'], item['task1_responses'][0]['Explanation'])
            #print(task1_responses)
            fr = item
            # Add responses to the new item
            fr['metric'] = json.loads(task1_responses[0].text.value)

            final_responses.append(fr)

    return final_responses

In [ ]:
process_json_file(data)

In [ ]:
import requests

# Define the endpoint and API version
endpoint = "https://culturalsnapsweden.openai.azure.com"
api_version = "2024-05-01"
api_key = GPT4V_KEY

# Construct the URL
url = f"{endpoint}/openai/models/gpt-4?api-version={api_version}"

# Define the headers
headers = {
    "api-key": api_key
}

# Make the GET request with headers
response = requests.get(url, headers=headers)
print(response._content)

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON response
    models = response.json()
    print(models)
else:
    print(f"Failed to retrieve models: {response.status_code}")
